### Импорт библиотек

In [1]:
from prepare_features import *
from download_data import *
from train import *
from submit_utils import *

### Загрузка данных

In [ ]:
download_new_data(INPUT_PATH, INPUT_PATH)
check_latest_date()

### Первоначальная обработка данных

In [ ]:
convert_timeseries_last_date(INPUT_PATH,FEATURES_PATH)

In [ ]:
convert_timeseries_confirmed(INPUT_PATH, FEATURES_PATH, 'confirmed', False)
convert_timeseries_confirmed(INPUT_PATH, FEATURES_PATH, 'deaths', False)

In [ ]:
for i in range(0, DAYS_TO_PREDICT + 1):
    create_time_features_confirmed(i, 'confirmed', FEATURES_PATH)
    create_time_features_confirmed(i, 'deaths', FEATURES_PATH)

In [ ]:
get_first_case_date('confirmed', INPUT_PATH, FEATURES_PATH)
get_first_case_date('deaths', INPUT_PATH, FEATURES_PATH)

### Обучение и предсказание

In [ ]:
start_time = time.time()
gbm_type = 'CatB'
params = get_params()
target = params['target']
id = params['id']
metric = params['metric']
limit_date = DAYS_TO_PREDICT

all_scores = dict()
alldays_preds_train = dict()
alldays_preds_test = dict()
for type in ['confirmed', 'deaths']:
    print('Go for type: {}'.format(type))
    alldays_preds_train[type] = []
    alldays_preds_test[type] = []
    for day in range(1, limit_date+1):
        train, test, features = read_input_data(day, type, step_back_days=STEP_BACK)
        print('Features: [{}] {}'.format(len(features), features))
        print('Test date: {}'.format(sorted(test['date'].unique())))

        if 0:
            train[features].to_csv(CACHE_PATH + 'train_debug_{}_{}.csv'.format(type, day))
            test[features].to_csv(CACHE_PATH + 'test_debug_{}_{}.csv'.format(type, day))

        overall_train_predictions, score, model_list, importance = create_catboost_model(train, features, params, day)
        prefix = '{}_day_{}_{}_{:.6f}'.format(gbm_type, day, len(model_list), score)
        save_in_file((score, model_list, importance, overall_train_predictions), MODELS_PATH + prefix + '.pklz')

        all_scores[(type, day)] = score
        train['pred'] = overall_train_predictions
        train['pred'] = np.maximum(train['pred'], train['case_day_minus_0'])
        train[['name1', 'name2', 'date', 'target', 'pred']].to_csv(SUBM_PATH + prefix + '_train.csv', index=False, float_format='%.8f')
        train_tmp = decrease_table_for_last_date(train)
        alldays_preds_train[type].append(train_tmp[['name1', 'name2', 'date', 'target', 'pred']].copy())

        overall_test_predictions = predict_with_catboost_model(test, features, model_list)
        test['pred'] = overall_test_predictions

        count_less = (test['pred'] < test['case_day_minus_0']).astype(np.int32).sum()
        if count_less > 0:
            print('Values less than needed for test: {} ({:.4f} %)'.format(count_less, 100 * count_less / len(test)))
        test['pred'] = np.maximum(test['pred'], test['case_day_minus_0'])

        test['shift_day'] = day
        test[['name1', 'name2', 'date', 'pred']].to_csv(SUBM_PATH + prefix + '_test.csv', index=False, float_format='%.8f')
        alldays_preds_test[type].append(test[['name1', 'name2', 'date', 'shift_day', 'pred']].copy())

    train = pd.concat(alldays_preds_train[type], axis=0)
    score = contest_metric(train['target'].values, train['pred'].values)
    all_scores[(type, 'full')] = score
    print('Total score {} for all days: {:.6f}'.format(type, score))
    prefix = '{}_{}_all_days_{}_{:.6f}'.format(gbm_type, type, len(model_list), score)
    train.to_csv(SUBM_PATH + '{}_train.csv'.format(prefix), index=False)
    test = pd.concat(alldays_preds_test[type], axis=0)
    test.to_csv(SUBM_PATH + '{}_test.csv'.format(prefix), index=False)

    prefix_2 = '{}_{}_LOG_{}_DIFF_{}_DIV_{}_countries'.format(gbm_type, type, USE_LOG, USE_DIFF, USE_DIV)
    train.to_csv(SUBM_PATH + '{}_train.csv'.format(prefix_2), index=False)
    test.to_csv(SUBM_PATH + '{}_test.csv'.format(prefix_2), index=False)

for type in ['confirmed', 'deaths']:
    for day in range(1, limit_date + 1):
        print('Type: {} Day: {} Score: {:.6f}'.format(type, day, all_scores[(type, day)]))
for type in ['confirmed', 'deaths']:
    print('Total score {} for all days: {:.6f}'.format(type, all_scores[(type, 'full')]))

print("Elapsed time overall: {:.2f} seconds".format((time.time() - start_time)))

### Создание финального предсказания

In [ ]:
subms = [
        [SUBM_PATH + 'CATB_confirmed_LOG_1_DIFF_1_DIV_1_countries_test.csv', SUBM_PATH + 'CATB_deaths_LOG_1_DIFF_1_DIV_1_countries_test.csv', 1]
]
score = 0.0
score = get_validation_score(subms)
ensemble(subms, score)